# A Recurrent Network Mechanism of Time Integration in Perceptual Decisions
Wong,  Wang
Journal of Neuroscience, January 2006, 26(4):1212-1328

## Material and Methods
### The full network model

### Calling Library Fuctions

In [1]:
# LIBRARY

import numpy as np # vector manipulation
import math  # math functions
import sys

# THIS IS FOR PLOTTING
%matplotlib inline
import matplotlib.pyplot as plt # side-stepping mpl backend
import warnings
warnings.filterwarnings("ignore")


### The Reduced Network Model
#### Firing rate
$$ \phi(I_{i})=\frac{c_{E,I}I_{syn}-I_{E,I}}{(1-exp(-g_{E,I}(C_{E,I}I_{syn}-I_{E,I}))}$$

Pyramidal cells: $I_E=125$Hz, $g_E=0.15$s, and $c_{E}=310(VnC)^{-1}$.

Interneurons: $I_I=177$Hz, $g_E=0.087$s, and $c_{I}=615(VnC)^{-1}$.

In [2]:
def PHI(c,I_syn,I,g):
    f=(c*I_syn-I)/(1-np.exp(-g*(g*I_syn-I)))
    return f

### Neural Circuit
$$ x_{1}=J_{11}S_1-J_{12}S_2+I_{0}+I_{1}+I_{noise,1}$$
$$ x_{2}=J_{22}S_2-J_{21}S_1+I_{0}+I_{2}+I_{noise,1}$$

where the synaptic couplings are $J_{11}=0.2609$, $J_{22}=0.2609$, $J_{12}=0.0497$ and $J_{21}=0.0497$.
$I_{0}=0.3255 nA$ represents external input 


In [3]:
def total_synaptic_current(S_1,S_2,I_1,I_2,I_noise_1,I_noise_2):
    # Synaptic coupling
    J_11=0.2609 # nA
    J_22=0.2609 # nA
    J_12=0.0497 # nA
    J_21=0.0497 # nA
    I_0=0.3255  # nA
    x_1=J_11*S_1-J_12*S_2+I_0+I_1+I_noise_1
    x_2=J_22*S_2-J_21*S_1+I_0+I_2+I_noise_2
    return x_1, x_2


### Background activity
$$ \tau_{AMPA}\frac{d I_{noise,i}(t)}{dt} =-I_{noise,i}(t)+\eta_i(t)\sqrt{\tau_{AMPA}}\sigma_{noise}$$

In [4]:
def Background_Activity(I_noise):
    h=0.2
    sigma_noise=0.02 # nA
    tau_AMPA=2 #ms
    eta_noise=np.random.normal(0,1,1)
    k=(-(I_noise)+eta_noise*np.sqrt(tau_AMPA)*sigma_noise)
    I_noise_new=I_noise+h/tau_AMPA*(-(I_noise+h/2*k)+eta_noise
                                *np.sqrt(tau_AMPA)*sigma_noise)
    return I_noise_new

### Network Dynamics
$$ \tau_r\frac{d r_{i}}{dt} =-r_{i}+\phi(I_{syn,i})$$

In [5]:
def Network_Dynamics(r,I):
    h=0.2 #ms
    tau_r=60 #ms
    r_new=r+h/tau_r*(-r+PHI(I))
    return r_new

$$ \tau_r\frac{d r_{I}}{dt} =-r_{I}+\phi(I_syn,I)$$

In [6]:
def r_I(r,I):
    h=0.2 #ms
    tau_r=60 #ms
    r_new=r+h/tau_r*(-r+PHI(I))
    return r_new

$$ \frac{d S_{AMPA,i}}{dt} =\frac{-S_{AMPA,i}}{\tau_{AMPA}}+r_i$$

In [7]:
def synamptic_AMPA(r,S):
    h=0.2 #ms
    tau_AMPA=60 #ms
    S_new=S+h(-S/tau_AMPA+r)
    return S_new

$$ \frac{d S_{NMDA,i}}{dt} =\frac{-S_{NMDA,i}}{\tau_{NMDA}}+(1-S_{NMDA,i})F(\psi(r_i))$$

In [8]:
def synamptic_NMDA(r,S):
    h=0.2 #ms
    tau_NMDA=60 #ms
    S_NMDA_new=S+h(-S/tau_NMDA+(1-S_NMDA)*F(psi(r)))
    return S_NMDA_new

$$ \frac{d S_{GABA,i}}{dt} =\frac{-S_{GABA,i}}{\tau_{GABA}}+r_i$$

In [9]:
def synamptic_GABA(r,S):
    h=0.2 #ms
    tau_GABA=60 #ms
    S_GABA_new=S+h(-S/tau_AMPA+r)
    return S_GABA_new

### Input Current Target

$$ I_i=J_{A,ext}\mu_0\left(1+ \frac{c'}{100} \right) $$


In [10]:
def I_input_1(c_dash):
    J_A_ext=0.00052 # nA/Hz
    mu_0=30 # Hz
    I_motion=J_A_ext*mu_0*(1+(c_dash)/100)
    return I_motion

$$ I_2=J_{A,ext}\mu_0\left(1- \frac{c'}{100} \right) $$

In [11]:
def I_input_2(c_dash):
    J_A_ext=0.00052 # nA/Hz
    mu_0=30 # Hz
    I_motion=J_A_ext*mu_0*(1-(c_dash)/100)
    return I_motion

In [12]:
h=0.2
time=np.arange(-1000,2000,h)
J_A_ext=0.00052 # nA/Hz
mu_0=30 # Hz

I_noise_1=0.04*np.random.normal(0,1,len(time))
I_noise_2=0.04*np.random.normal(0,1,len(time))

x_1=0*J_A_ext*mu_0*np.random.uniform(0,1,len(time))
x_2=0*J_A_ext*mu_0*np.random.uniform(0,1,len(time))

S_1=0.1*np.random.uniform(0,1,len(time))# np.random.normal(0,1,len(time))*0.0004
S_2=0.1*np.random.uniform(0,1,len(time)) #np.random.normal(0,1,len(time))*0.0004

I_1=0.0*np.ones(len(time)) # np.zeros((1,len(time)))
I_2=0.0*np.ones(len(time)) # np.zeros((1,len(time)))
Firing_target_1=0*time # np.zeros((1,len(time)))
Firing_target_2=0*time # np.zeros((1,len(time)))




In [14]:
c_dash=0.0

for i in range (0,len(time)-1):
    if time[i] >=500 and time[i]<600:
        c_dash=25.0
    elif time[i] >=700 and time[i]<800:
        c_dash=35.0
    elif time[i] >=900 and time[i]<1000:
        c_dash=45.0
    elif time[i] >=1100 and time[i]<1200:
        c_dash=25.0
    else:
        c_dash=0.0
    
    
    Firing_target_1[i]=PHI(x_1[i])
    Firing_target_2[i]=PHI(x_2[i])
    
    I_noise_1[i+1]=Background_Activity(I_noise_1[i])
    I_noise_2[i+1]=Background_Activity(I_noise_2[i])
    
    I_1[i+1]=I_input_1(c_dash)
    I_2[i+1]=I_input_2(c_dash)
   
    x_1[i+1],x_2[i+1]=total_synaptic_current(S_1[i],S_2[i],I_1[i+1],
                                      I_2[i+1],I_noise_1[i+1],I_noise_2[i+1])
    S_1[i+1]=Network_Dynamics(S_1[i],x_1[i+1])
    S_2[i+1]=Network_Dynamics(S_2[i],x_2[i+1])
   


TypeError: PHI() missing 3 required positional arguments: 'I_syn', 'I', and 'g'

### Plotting
#### Input

In [ ]:
fig = plt.figure(figsize=(8,4))
plt.subplot(311)
plt.plot(time,S_1,'-',color='blue',label="S_1")
plt.plot(time,S_2,'-',color='red',label="S_2")
plt.grid()
plt.xlim((-500,1500))
plt.xlabel('time(ms)')
plt.ylabel('S')
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)

plt.subplot(312)
plt.plot(time,Firing_target_1,'-',color='blue',label="H_1")
plt.plot(time,Firing_target_2,'-',color='red',label="H_2")
plt.grid()
plt.xlabel('time(ms)')
plt.ylabel('Firing Rate (Hz)')
plt.xlim((-500,1500))
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)

plt.subplot(313)
plt.plot(time,I_1,'-',color='blue',label="I_1")
plt.plot(time,I_1,'-',color='red',label="I_2")
plt.grid()
plt.xlabel('time(ms)')
plt.ylabel('Input')
plt.xlim((-500,1500))
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)

#plt.ylim((30,50))
plt.show()

fig = plt.figure(figsize=(8,4))
plt.plot(S_1,S_2,'-',color='red')
plt.grid()
plt.xlim((-500,1500))
plt.xlabel('S1')
plt.ylabel('S2')
plt.title('Phase Plane')
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)

